In [63]:
workspace()
import Base.^
import Base.*
import Base.convert
import Base.promote_rule

# Zad 1

In [64]:
function printTree(x)
    if(isa(x, Type{Any}))
        return x
    end
    return string(printTree(supertype(x)),"-->",x)
end

printTree (generic function with 1 method)

In [65]:
printTree(Float16)

"Any-->Number-->Real-->AbstractFloat-->Float16"

# Zad 2
## Definicja grupy, konstruktor

In [66]:
struct Gn{N}
           x::Integer
           Gn{N}(x) where N = new(getGroupMember(x,N))
       end
function getGroupMember(x,N)
    if(x>=N)
        x=x%N
    end
    if(gcd(x,N)!=1 && x>0)
        throw(DomainError())
    end
    return x
end


getGroupMember (generic function with 1 method)

In [67]:
g1=Gn{5}(7)

Gn{5}(2)

In [68]:
g1.x

2

## Mnożenie

In [69]:
function *{N}(A::Gn{N}, B::Gn{N})
    return Gn{N}(A.x*B.x)
  end
function *{N}(A::Gn{N}, B::Integer)
    return Gn{N}(A.x*B)
  end
function *{N}(A::Integer,B::Gn{N})
    return B*A
  end

* (generic function with 188 methods)

In [70]:
g1*g1

Gn{5}(4)

In [71]:
g1*7

Gn{5}(4)

In [72]:
7*g1

Gn{5}(4)

## Funkcja konwertująca

In [73]:
convert{N}(::Type{Gn{N}}, x::Int64) = Gn{N}(x)
convert{N}(::Type{Int64}, x::Gn{N}) = x.x

convert (generic function with 725 methods)

In [74]:
convert(Gn{5},10)

Gn{5}(0)

In [75]:
convert(Int64,g1)

2

## Promocja

In [76]:
promote_rule(::Type{Gn{N}}, ::Type{T}) where {T<:Integer,N} = Gn{N}

promote_rule (generic function with 125 methods)

In [77]:
promote_type(Gn{5}, Int32)

Gn{5}

## Potęgowanie

In [78]:
function ^{N}(A::Gn{N},B::Integer)
    x=Gn{N}(1)
    for i = 1:B
           x=x*A
       end
    return x
  end

^ (generic function with 54 methods)

In [79]:
g2=Gn{1000}(7)

Gn{1000}(7)

In [80]:
g2^3

Gn{1000}(343)

In [81]:
7^3

343

In [82]:
g1^3

Gn{5}(3)

In [83]:
7^3%5

3

## Okres liczby

In [84]:
function period{N}(A::Gn{N})
    return filter(x->(try convert(Int64,(A^x)) catch 0  end ==1) ,1:N)[1]
end

period (generic function with 1 method)

In [85]:
period(g2)

20

## Element odwrotny

In [86]:
function inverse{N}(A::Gn{N})
    return filter(x->try convert(Int64,(A*x)) catch 0 end ==1,1:N)[1]
end

inverse (generic function with 1 method)

In [87]:
inverse(g2)

143

In [88]:
g2

Gn{1000}(7)

In [89]:
7*143

1001

## Ilość elementów

In [90]:
function count{N}(T::Type{Gn{N}})
    return length(filter(x->try convert(Int64,Gn{N}(x))==x catch false end,1:N))
end

count (generic function with 1 method)

In [91]:
count(Gn{6}) # 1 5

2

In [92]:
count(Gn{5}) # 1 2 3 4

4

## łamanie RSA

In [93]:
N=55 #public
c=17 #public
b=4; #message

In [94]:
r=period(Gn{N}(b))

10

In [95]:
d=inverse(Gn{r}(c))

3

In [96]:
a=Gn{N}(b)^d

Gn{55}(9)

In [100]:
b==(Gn{N}(convert(Int64,a))^c).x

true

## Szyfrowanie RSA
https://en.wikipedia.org/wiki/RSA_(cryptosystem)

### Generating keys

In [1]:
p=61
q=53
n=p*q

3233

In [547]:
k=(p-1)*(q-1)

3120

In [548]:
e=17

17

In [549]:
d=inverse(Gn{k}(e))

2753

### encrypting and decrypting

In [550]:
m=65

65

In [551]:
c=(Gn{n}(m)^e).x

2790

In [552]:
m_dec=(Gn{n}(c)^d).x

65

In [553]:
m_dec==m

true